In [42]:
import pandas as pd
import os
import numpy as np
import pickle

from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.func_demograficas import *

In [2]:
def codif_diada_depto(df, origen, destino):
    "Codifica díadas por departamento"
    cod_depto_destino_str = df[destino].astype(str).str.zfill(2)
    cod_diada_depto = (df[origen].astype(str) + cod_depto_destino_str).astype(int)

    return cod_diada_depto


def agrupar_por_tramos(df, col_tramo, col_edad, col_sexo):
    """
    Esta función agrupa por tramos, edad y sexo
    un pandas dataframe con datos del Censo 2011
    para ser usado para graficar pirámides de población
    """  
    # genera lista con cortes, para reclasificar el dataframe
    bins = [0 if i==-1 else i for i in range(-1,95,5)]

    # etiquetas con los tramos de edad usuales en pirámides de población
    labels = ['0-4', '5-9', '10-14', '15-19', '20-24',
              '25-29', '30-34', '35-39', '40-44', '45-49',
             '50-54', '55-59', '60-64', '65-69', '70-74',
              '75-79', '80-84', '85-89', '90-94', '+95']

    # calcula tramos de edad
    df[col_tramo] = pd.cut(df[col_edad],
                           bins=bins,
                           include_lowest=False,
                           labels=labels[:19])
    df[col_tramo] = np.where(df[col_edad] > 94 , labels[-1], df[col_tramo])
    df[col_tramo] = np.where(df[col_edad] == 0 ,  labels[0], df[col_tramo])

    # agrupa
    df_group = df.groupby([col_sexo, col_tramo]).size().reset_index()
    
    # renombra vars
    df_group.rename(columns={col_sexo: 'sexo', 0:'personas'}, inplace=True)
    
    # calcula porcentajes
    df_group['porc_pers'] = (df_group.personas / df_group.personas.sum())*100
    
    # pasa varones a negativo para grafica a la izquierda del eje central
    df_group.loc[df_group.sexo == 1, 'personas'] = -df_group['personas']
    df_group.loc[df_group.sexo == 1, 'porc_pers'] = -df_group['porc_pers']
    
    # genera etiqueta del sexo
    df_group['sexo_label'] = np.where(df_group['sexo'] ==1, 'varones', 'mujeres')
    
    return df_group

In [3]:
mig = cargar_migrantes_internos()
mig.columns

Index(['depto_origen', 'depto_destino', 'LOC', 'SECC', 'SEGM', 'VIVID',
       'TIPO_VIVIE', 'HOGCOD', 'HOGID', 'PERPH02', 'PERPA01', 'PERER02',
       'PERNA01', 'PERNA02', 'PERMI01', 'PERMI01_1', 'PERMI01_2', 'PERMI01_3',
       'PERMI01_4', 'PERMI02', 'PERMI02_1', 'PERMI05', 'PERMI05_1', 'PERMI06',
       'PERMI06_1', 'PERMI06_2', 'PERMI06_3', 'PERMI06_4', 'PERMI07',
       'PERMI07_1', 'PERMI07_2', 'PERMI07_3', 'PERMI07_4', 'PERED00',
       'PERED01', 'PERED02', 'PERED02_1', 'PERED02_2', 'PERED02_3',
       'PERED02_4', 'PERED03_R', 'PERED03_1', 'PERED03_2', 'PERED04_R',
       'PERED05_R', 'PERED06_R', 'CODIGO_CAR', 'PERED08', 'NIVELEDU_R',
       'loc_origen', 'loc_destino'],
      dtype='object')

In [4]:
mig['cod'] = codif_diada_depto(mig, 'depto_origen', 'depto_destino')
mig.head(3)

,depto_origen,depto_destino,LOC,SECC,SEGM,VIVID,TIPO_VIVIE,HOGCOD,HOGID,PERPH02,...,PERED03_2,PERED04_R,PERED05_R,PERED06_R,CODIGO_CAR,PERED08,NIVELEDU_R,loc_origen,loc_destino,cod
0,3,1,20,1,1,5,1,102001001000501,1,1,...,0,2,3,4,55131,1,9,3732,1020,301
1,17,1,20,1,1,30,1,102001001003001,1,2,...,0,0,2,4,55152,1,9,17220,1020,1701
2,18,1,20,1,1,36,1,102001001003601,1,1,...,0,2,2,0,33101,1,5,18220,1020,1801


In [5]:
def agrupador(df, grouper):
    "Agrupa datos censales"
    mig_data = df.groupby(grouper) \
      .apply(lambda x: pd.Series({
          'n':             x['PERPH02'].count(),
          'edad_promedio': x['PERNA01'].mean(),
          'edad_mediana':  x['PERNA01'].median(),
          'varones':  x['PERPH02'].loc[x['PERPH02']==1].count(),
          'mujeres':  x['PERPH02'].loc[x['PERPH02']==2].count(),
      })
    ).reset_index()

    # cambia a integers
    cols = ['n', 'edad_mediana', 'varones', 'mujeres']
    mig_data.loc[:, cols] = mig_data.loc[:, cols].astype(int)

    # agrega índice de masculinidad
    im = (mig_data.loc[:, 'varones'] / mig_data.loc[:, 'mujeres'])

    mig_data['ind_masc'] = round(im*100, 2)

    return mig_data

In [6]:
mig_data = agrupador(mig, ['depto_origen', 'depto_destino', 'cod'])
mig_data.head()

,depto_origen,depto_destino,cod,n,edad_promedio,edad_mediana,varones,mujeres,ind_masc
0,1,2,102,914,30.855580,28,428,486,88.07
1,1,3,103,33127,35.474749,33,16049,17078,93.97
2,1,4,104,1387,32.630137,30,684,703,97.30
3,1,5,105,2100,34.265238,32,944,1156,81.66
4,1,6,106,982,31.549898,30,454,528,85.98


In [7]:
censo =  cargar_censo()
censo['group'] = 1
agg_censo = agrupador(censo, 'group')
agg_censo

,group,n,edad_promedio,edad_mediana,varones,mujeres,ind_masc
0,1,3285877,36.13446,34,1577416,1708461,92.33


In [28]:
cods = list(set(mig.cod.values))
len(cods)

342

In [79]:
agru = []

for i in cods:
    data = mig.loc[mig.cod==i].copy()
    ag = agrupar_por_tramos(data, 'tramo', 'PERNA01', 'PERPH02')
    ag['cod'] = i
    agru.append(ag)
    
datos_piramides = pd.concat(agru)
datos_piramides.head()

,sexo,tramo,personas,porc_pers,sexo_label,cod
0,1,10-14,-42,-4.595186,varones,102
1,1,15-19,-29,-3.172867,varones,102
2,1,20-24,-25,-2.735230,varones,102
3,1,25-29,-67,-7.330416,varones,102
4,1,30-34,-58,-6.345733,varones,102


In [77]:
datos_piramides.to_pickle('tablas/agrup_piramides_tablero.pkl', compression='gzip')

In [78]:
datos_piramides.to_csv('tablas/agrup_piramides_tablero.csv', index=False)

## 

In [54]:
# exports data
mig_data.to_csv('tablas/datos_tablero.csv', index=False)


os.chdir('/home/guillermo/Documentos/GitHub/migraciones_internas/data/')
mig_data.to_csv('datos_tablero.csv', index=False)